In [1]:
pip install pyspark

     |████████████████████████████████| 212.3MB 69kB/s 
     |████████████████████████████████| 204kB 55.9MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=dc684d3c1dbe15d333df387434e7240544bd5231d39b67ac73ad57b61560c808
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [3]:
import time
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

In [8]:
import pandas as pd
df = spark.read.csv('machine.data')
df.show()

+---------+--------+---+-----+-----+---+---+---+----+----+
|      _c0|     _c1|_c2|  _c3|  _c4|_c5|_c6|_c7| _c8| _c9|
+---------+--------+---+-----+-----+---+---+---+----+----+
|  adviser|   32/60|125|  256| 6000|256| 16|128| 198| 199|
|   amdahl|  470v/7| 29| 8000|32000| 32|  8| 32| 269| 253|
|   amdahl| 470v/7a| 29| 8000|32000| 32|  8| 32| 220| 253|
|   amdahl| 470v/7b| 29| 8000|32000| 32|  8| 32| 172| 253|
|   amdahl| 470v/7c| 29| 8000|16000| 32|  8| 16| 132| 132|
|   amdahl|  470v/b| 26| 8000|32000| 64|  8| 32| 318| 290|
|   amdahl|580-5840| 23|16000|32000| 64| 16| 32| 367| 381|
|   amdahl|580-5850| 23|16000|32000| 64| 16| 32| 489| 381|
|   amdahl|580-5860| 23|16000|64000| 64| 16| 32| 636| 749|
|   amdahl|580-5880| 23|32000|64000|128| 32| 64|1144|1238|
|   apollo|   dn320|400| 1000| 3000|  0|  1|  2|  38|  23|
|   apollo|   dn420|400|  512| 3500|  4|  1|  6|  40|  24|
|     basf|    7/65| 60| 2000| 8000| 65|  1|  8|  92|  70|
|     basf|    7/68| 50| 4000|16000| 65|  1|  8| 138| 11

In [9]:
names = ['vendor name','Model name', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'ERP']

In [ ]:
k = df.columns
old = df.schema.names
for i in range(len(df.columns)):
  df = df.withColumnRenamed(old[i],names[i])

In [17]:
df = df.na.drop()
df.show()

+-----------+----------+----+-----+-----+----+-----+-----+----+----+
|vendor name|Model name|MYCT| MMIN| MMAX|CACH|CHMIN|CHMAX| PRP| ERP|
+-----------+----------+----+-----+-----+----+-----+-----+----+----+
|    adviser|     32/60| 125|  256| 6000| 256|   16|  128| 198| 199|
|     amdahl|    470v/7|  29| 8000|32000|  32|    8|   32| 269| 253|
|     amdahl|   470v/7a|  29| 8000|32000|  32|    8|   32| 220| 253|
|     amdahl|   470v/7b|  29| 8000|32000|  32|    8|   32| 172| 253|
|     amdahl|   470v/7c|  29| 8000|16000|  32|    8|   16| 132| 132|
|     amdahl|    470v/b|  26| 8000|32000|  64|    8|   32| 318| 290|
|     amdahl|  580-5840|  23|16000|32000|  64|   16|   32| 367| 381|
|     amdahl|  580-5850|  23|16000|32000|  64|   16|   32| 489| 381|
|     amdahl|  580-5860|  23|16000|64000|  64|   16|   32| 636| 749|
|     amdahl|  580-5880|  23|32000|64000| 128|   32|   64|1144|1238|
|     apollo|     dn320| 400| 1000| 3000|   0|    1|    2|  38|  23|
|     apollo|     dn420| 400|  512

In [18]:
df.printSchema()

root
 |-- vendor name: string (nullable = true)
 |-- Model name: string (nullable = true)
 |-- MYCT: string (nullable = true)
 |-- MMIN: string (nullable = true)
 |-- MMAX: string (nullable = true)
 |-- CACH: string (nullable = true)
 |-- CHMIN: string (nullable = true)
 |-- CHMAX: string (nullable = true)
 |-- PRP: string (nullable = true)
 |-- ERP: string (nullable = true)



In [19]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(df) for column in list(set(df.columns)-set(['date'])) ]


pipeline = Pipeline(stages=indexers)
df_r = pipeline.fit(df).transform(df)

df_r.show()


+-----------+----------+----+-----+-----+----+-----+-----+----+----+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+
|vendor name|Model name|MYCT| MMIN| MMAX|CACH|CHMIN|CHMAX| PRP| ERP|MMAX_index|MMIN_index|CACH_index|MYCT_index|PRP_index|CHMIN_index|Model name_index|CHMAX_index|vendor name_index|ERP_index|
+-----------+----------+----+-----+-----+----+-----+-----+----+----+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+
|    adviser|     32/60| 125|  256| 6000| 256|   16|  128| 198| 199|       7.0|       5.0|      14.0|      23.0|     69.0|        5.0|            29.0|       19.0|             26.0|     66.0|
|     amdahl|    470v/7|  29| 8000|32000|  32|    8|   32| 269| 253|       3.0|       4.0|       2.0|      18.0|     76.0|        2.0|            62.0|        3.0|              6.0|     16.0|
|     amdahl|   470v/7a|  29| 8000|32000

In [39]:
type(df_r)

pyspark.sql.dataframe.DataFrame

In [41]:
names_index = []
for name in names:
  names_index.append(name+'_index')

In [43]:
names_index

['vendor name_index',
 'Model name_index',
 'MYCT_index',
 'MMIN_index',
 'MMAX_index',
 'CACH_index',
 'CHMIN_index',
 'CHMAX_index',
 'PRP_index',
 'ERP_index']

In [45]:
from pyspark.ml.feature import VectorAssembler
assemble=VectorAssembler(inputCols=names_index, outputCol='features')
assembled_data=assemble.transform(df_r)
assembled_data.show()

+-----------+----------+----+-----+-----+----+-----+-----+----+----+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+--------------------+
|vendor name|Model name|MYCT| MMIN| MMAX|CACH|CHMIN|CHMAX| PRP| ERP|MMAX_index|MMIN_index|CACH_index|MYCT_index|PRP_index|CHMIN_index|Model name_index|CHMAX_index|vendor name_index|ERP_index|            features|
+-----------+----------+----+-----+-----+----+-----+-----+----+----+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+--------------------+
|    adviser|     32/60| 125|  256| 6000| 256|   16|  128| 198| 199|       7.0|       5.0|      14.0|      23.0|     69.0|        5.0|            29.0|       19.0|             26.0|     66.0|[26.0,29.0,23.0,5...|
|     amdahl|    470v/7|  29| 8000|32000|  32|    8|   32| 269| 253|       3.0|       4.0|       2.0|      18.0|     76.0|        2.0|            62

In [46]:
from pyspark.ml.feature import StandardScaler
scale=StandardScaler(inputCol='features',outputCol='standardized')
data_scale=scale.fit(assembled_data)
data_scale_output=data_scale.transform(assembled_data)
data_scale_output.show(2)

+-----------+----------+----+----+-----+----+-----+-----+---+---+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+--------------------+--------------------+
|vendor name|Model name|MYCT|MMIN| MMAX|CACH|CHMIN|CHMAX|PRP|ERP|MMAX_index|MMIN_index|CACH_index|MYCT_index|PRP_index|CHMIN_index|Model name_index|CHMAX_index|vendor name_index|ERP_index|            features|        standardized|
+-----------+----------+----+----+-----+----+-----+-----+---+---+----------+----------+----------+----------+---------+-----------+----------------+-----------+-----------------+---------+--------------------+--------------------+
|    adviser|     32/60| 125| 256| 6000| 256|   16|  128|198|199|       7.0|       5.0|      14.0|      23.0|     69.0|        5.0|            29.0|       19.0|             26.0|     66.0|[26.0,29.0,23.0,5...|[3.58043973945428...|
|     amdahl|    470v/7|  29|8000|32000|  32|    8|   32|269|253|       3.0|

In [47]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
silhouette_score=[]
evaluator = ClusteringEvaluator(predictionCol='prediction', featuresCol='standardized', \
                                metricName='silhouette', distanceMeasure='squaredEuclidean')
for i in range(2,10):
    
    KMeans_algo=KMeans(featuresCol='standardized', k=i)
    
    KMeans_fit=KMeans_algo.fit(data_scale_output)
    
    output=KMeans_fit.transform(data_scale_output)
    
    
    
    score=evaluator.evaluate(output)
    
    silhouette_score.append(score)
    
    print("Silhouette Score:",score)

Silhouette Score: 0.29616971068845677
Silhouette Score: 0.2709272139151995
Silhouette Score: 0.22507478494676675
Silhouette Score: 0.2166552132362425
Silhouette Score: 0.20818952608037886
Silhouette Score: 0.18739476654334564
Silhouette Score: 0.19649345343138194
Silhouette Score: 0.22569118188284598
